In [2]:
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
from google.cloud import texttospeech
from pydub import AudioSegment
import simpleaudio as sa
import tempfile

ImportError: cannot import name 'texttospeech' from 'google.cloud' (unknown location)

In [ ]:

# Voice and language dropdowns
lang_dropdown = widgets.Dropdown(
    options=[('English (US)', 'en-US'), ('Irish (IE)', 'ga-IE')],
    value='en-US',
    description='Language:'
)

gender_dropdown = widgets.Dropdown(
    options=['FEMALE', 'MALE', 'NEUTRAL'],
    value='FEMALE',
    description='Gender:'
)

# SSML input area
ssml_input = widgets.Textarea(
    value="<speak>Hello world!</speak>",
    placeholder='Paste SSML here (including IPA)',
    description='SSML:',
    layout=widgets.Layout(width='100%', height='150px')
)

# Output area
output_area = widgets.Output()

# Play button
play_button = widgets.Button(
    description='Play',
    button_style='success',
    tooltip='Generate and play TTS audio',
    icon='play'
)

# TTS playback function
def synthesize_and_play_ssml(ssml_text, lang_code, gender):
    client = texttospeech.TextToSpeechClient()

    synthesis_input = texttospeech.SynthesisInput(ssml=ssml_text)
    voice_params = texttospeech.VoiceSelectionParams(
        language_code=lang_code,
        ssml_gender=getattr(texttospeech.SsmlVoiceGender, gender)
    )
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.LINEAR16)

    response = client.synthesize_speech(
        input=synthesis_input,
        voice=voice_params,
        audio_config=audio_config
    )

    temp_wav = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    temp_wav.write(response.audio_content)
    temp_wav.close()

    audio = AudioSegment.from_wav(temp_wav.name)
    sa.play_buffer(audio.raw_data, num_channels=audio.channels,
                   bytes_per_sample=audio.sample_width, sample_rate=audio.frame_rate)

    return Audio(temp_wav.name)

# Button click callback
def on_play_clicked(b):
    with output_area:
        clear_output(wait=True)
        ssml_text = ssml_input.value
        lang_code = lang_dropdown.value
        gender = gender_dropdown.value
        try:
            audio_widget = synthesize_and_play_ssml(ssml_text, lang_code, gender)
            display(audio_widget)
        except Exception as e:
            print(f"❌ Error: {e}")

play_button.on_click(on_play_clicked)

# Display UI
ui = widgets.VBox([
    widgets.HBox([lang_dropdown, gender_dropdown]),
    ssml_input,
    play_button,
    output_area
])

display(ui)
